In [38]:
EG = "/Users/joregan/Playing/merged_annotations/hsi_4_0717_209_002.json"

In [39]:
import json

In [40]:
with open(EG) as inf:
    data = json.load(inf)

In [9]:
SAMPLE = """
{
  "2": {
    "general": {
      "id": 2,
      "start": 5.970287890159987,
      "end": 9.551266621561616
    },
    "high_level": {
      "topic_name": "reference_real",
      "current_topic": "vase",
      "topic_change": false,
      "topic_duration_id": 0,
      "spatial_reference": "fourth vase",
      "referenced_object": [
        "vase"
      ]
    },
    "low_level": {
      "resolved_references": {
        "the fourth vase": "vase",
        "it": "vase"
      },
      "spatial_relationships": [],
      "resolved_adverbs": []
    },
    "snippet": "But I'm wondering about the fourth vase. Where is it? Did you break it?"
  }
}
"""

```json
      "resolved_references_indices": {
        "it": [2, 4]
      }
```

In [ ]:
def load_tsv(filename):
    data = []
    with open(filename) as inf:
        for line in inf.readlines():
            parts = line.strip().split("\t")
            data.append({
                "start": float(parts[0]),
                "end": float(parts[1]),
                "word": parts[2]
            })
    return data

In [47]:
def slice_tsv_data(data, start, end):
    ret = []
    for datum in data:
        if type(datum["start"]) is str:
            datum["start"] = float(datum["start"])
        if type(datum["end"]) is str:
            datum["end"] = float(datum["end"])
        if datum["start"] >= start and datum["end"] <= end:
            ret.append(datum)
        elif datum["end"] > end:
            return ret
    return ret

In [50]:
def clean_text(text):
    return " ".join([x.lower().strip(".,;?!") for x in text.split(" ")])

In [51]:
from pathlib import Path

TSVS = Path("/Users/joregan/Playing/hsi/word_annotations/")
JSON = Path("/Users/joregan/Playing/merged_annotations/")

In [93]:
def get_indices(needle, haystack, checkpos=True):
    ret = []
    nwords = [x.lower().strip(",?.;:") for x in needle.split(" ")]
    hwords = [x.lower().strip(",?.;:") for x in haystack.split(" ")]
    nwordspos = nwords[:-1] + [f"{nwords[-1]}'s"]
    nlen = len(nwords)

    for i in range(len(hwords)):
        if hwords[i:i+nlen] == nwords:
            ret.append((i, i+nlen))
        elif checkpos and hwords[i:i+nlen] == nwordspos:
            ret.append((i, i+nlen))
    return ret

In [ ]:
def get_tsv_for_segment(segment, tsv_data):
    assert "general" in segment, "Missing key 'general'"
    assert "start" in segment["general"], "Missing key 'start'"
    assert "end" in segment["general"], "Missing key 'end'"

    start = segment["general"]["start"]
    end = segment["general"]["end"]

    tsv = slice_tsv_data(tsv_data, start, end)
    tsv_words = " ".join([x["word"] for x in tsv])

    if segment["snippet"] != tsv_words:
        cleaned_snippet = clean_text(segment["snippet"])
        cleaned_text = clean_text(tsv_words)

        if cleaned_snippet not in cleaned_text:
            print("🙀 mismatch:", "🖇️", segment["snippet"], "🎧", tsv_words, cleaned_text.find(cleaned_snippet))
            return []
        else:
            idxes = get_indices(cleaned_snippet, cleaned_text)
            assert len(idxes) == 1
            tsv = tsv[idxes[0][0]:idxes[0][1]]
            tsv_words = " ".join([x["word"] for x in tsv])
            cleaned_text = clean_text(tsv_words)
            assert cleaned_snippet == cleaned_text, f"🖇️ {cleaned_snippet} 🎧 {cleaned_text}"
    return tsv

In [113]:
def is_skippable(segment, strict=True):
    skippables = ["conversation_generic"]
    if strict:
        skippables += ["reference_imaginary"]
    if segment["high_level"]["topic_name"] == "reference_unreal":
        segment["high_level"]["topic_name"] = "reference_imaginary"
    if segment["high_level"]["topic_name"] in skippables:
        return True
    elif segment["low_level"]["resolved_references"] == {}:
        return True
    else:
        return False

In [ ]:
# that weird waste bin [(1, 5)]
# that [(1, 2), (8, 9)]
def skip_overlapped_index(a, b):
    if a[0] >= b[0] and a[1] <= b[1]:
        return True
    return False

assert skip_overlapped_index((1, 2), (1, 5)) == True
assert skip_overlapped_index((1, 5), (1, 2)) == False

In [143]:
def prune_manual_index(indices, manual):
    ret = []
    for index in indices:
        if index[0] in manual:
            ret.append(index)
    return ret

assert prune_manual_index([(1, 3), (5, 7)], [1]) == [(1, 3)]
assert prune_manual_index([(1, 3), (5, 7)], [1, 5]) == [(1, 3), (5, 7)]

In [144]:
def process_segment(segment, tsv_data):
    if is_skippable(segment):
        return
    tsv = get_tsv_for_segment(segment, tsv_data)
    references = segment["low_level"]["resolved_references"]
    manual_idx = segment["low_level"].get("resolved_references_indices", {})

    # these are ordered. Kinda.
    indices = {}
    for ref in references:
        indices[ref] = get_indices(ref, segment["snippet"])
        print("1", ref, indices[ref])
        if ref in manual_idx:
            indices[ref] = prune_manual_index(indices[ref], manual_idx[ref])
        # for index in indices:
        #     print(ref, index, tsv[index[0]:index[1]])
        print("2", ref, indices[ref])

    # print(references)

In [145]:
for seg in data:
    print(seg)
    print(process_segment(data[seg], ttsv))

0
None
1
None
2
None
3
None
4
None
5
1 See-through [(0, 1)]
2 See-through [(0, 1)]
1 blue [(1, 2)]
2 blue [(1, 2)]
1 green [(2, 3)]
2 green [(2, 3)]
None
6
None
7
None
8
None
9
1 that weird waste bin [(1, 5)]
2 that weird waste bin [(1, 5)]
1 that [(1, 2), (8, 9)]
2 that [(1, 2), (8, 9)]
None
10
1 that [(3, 4)]
2 that [(3, 4)]
1 it [(12, 13)]
2 it [(12, 13)]
None
11
1 it [(2, 3)]
2 it [(2, 3)]
None
12
1 the desk [(8, 10)]
2 the desk [(8, 10)]
1 it [(11, 12), (15, 16)]
2 it [(11, 12), (15, 16)]
None
13
None
14
None
15
1 the table [(16, 18)]
2 the table [(16, 18)]
None
16
1 it [(3, 4)]
2 it [(3, 4)]
None
17
None
18
1 it [(13, 14)]
2 it [(13, 14)]
None
19
None
20
1 the couches [(2, 4)]
2 the couches [(2, 4)]
None
21
None
22
1 them [(3, 4)]
2 them [(3, 4)]
None
23
None
24
None
25
1 the windows [(0, 2)]
2 the windows [(0, 2)]
None
26
None
27
None
28
None
29
1 them [(4, 5)]
2 them [(4, 5)]
None
30
None
31
None
32
None
33
None
34
1 the fireplace [(4, 6)]
2 the fireplace [(4, 6)]
None
35
None


In [141]:
data["57"]

{'general': {'id': 57, 'start': 216.59330501354725, 'end': 219.9003740473728},
 'high_level': {'topic_name': 'reference_real',
  'current_topic': 'lamp',
  'topic_change': False,
  'topic_duration_id': 3,
  'spatial_reference': '',
  'referenced_object': ['lamp']},
 'low_level': {'resolved_references': {'it': 'lamp'},
  'spatial_relationships': [],
  'resolved_adverbs': [],
  'resolved_references_indices': {'it': [2, 10]}},
 'snippet': "It's like, it looks weird. Did you do something with it?"}